In [5]:
import logging
from eppy.modeleditor import IDF
import glob
import pandas as pd
import numpy as np
import pyumi as pu

In [6]:
files = glob.glob("data/necb/NECB_2011_Montreal_idf/*.idf")
files

['data/necb/NECB_2011_Montreal_idf\\NECB 2011-FullServiceRestaurant-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.idf',
 'data/necb/NECB_2011_Montreal_idf\\NECB 2011-HighriseApartment-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.idf',
 'data/necb/NECB_2011_Montreal_idf\\NECB 2011-Hospital-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.idf',
 'data/necb/NECB_2011_Montreal_idf\\NECB 2011-PrimarySchool-NECB HDD Method-CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw.idf']

In [7]:
logging.info("start")

In [8]:
idfs = []
# Windows
iddfile = "C:\openstudio-2.5.2\EnergyPlus\Energy+.idd"
# Mac
# iddfile = "/Applications/OpenStudio-2.5.0/EnergyPlus/Energy+.idd"
epw = 'CAN_PQ_Montreal.Intl.AP.716270_CWEC.epw'
IDF.setiddname(iddfile)
for file in files:
    idfs.append(IDF(file, epw))

In [9]:
keys = [idf.idfobjects['BUILDING'].list2[0][1] for idf in idfs]

## GasMaterials

In [10]:
GasMaterials = pu.parse_idfs(idfs, 'WINDOWMATERIAL:GAS', keys)

In [11]:
GasMaterials['Cost'] = 0
GasMaterials['EmbodiedCarbon'] = 0
GasMaterials['EmbodiedCarbonStdDev'] = 0
GasMaterials['EmbodiedEnergy'] = 0
GasMaterials['EmbodiedEnergyStdDev'] = 0
GasMaterials['SubstitutionRatePattern'] = np.NaN # ! Might have to change to an empty array
GasMaterials['SubstitutionTimestep'] = 0
GasMaterials['TransportCarbon'] = 0
GasMaterials['TransportDistance'] = 0
GasMaterials['TransportEnergy'] = 0

In [12]:
def gas_type(row):
    if 'air' in row['Gas_Type'].lower():
        return 0
    elif 'argon' in row['Gas_Type']:
        return 1
    elif 'krypton' in row['Gas_Type'].lower():
        return 2
    elif 'xenon' in row['Gas_Type'].lower():
        return 3
    elif 'sf6' in row['Gas_Type'].lower():
        return 4

In [13]:
# Add GasType Column based on previous function
GasMaterials['GasType'] = GasMaterials.apply(lambda x: gas_type(x), axis=1)

In [14]:
# Remove unnecessary columns
GasMaterials.drop(['key','Gas_Type', 'Thickness'], axis=1, inplace=True)

In [15]:
GasMaterials

,Name,Archetype,Cost,EmbodiedCarbon,EmbodiedCarbonStdDev,EmbodiedEnergy,EmbodiedEnergyStdDev,SubstitutionRatePattern,SubstitutionTimestep,TransportCarbon,TransportDistance,TransportEnergy,GasType
$id,,,,,,,,,,,,,
0,AIR 13MM,NECB 2011-FullServiceRestaurant-NECB HDD Metho...,0,0,0,0,0,NaN,0,0,0,0,0
1,AIR 6MM,NECB 2011-FullServiceRestaurant-NECB HDD Metho...,0,0,0,0,0,NaN,0,0,0,0,0


In [16]:
def newrange(previous, following):
    From = previous.iloc[[-1]].index.values + 1
    To = From + len(following)
    return np.arange(From, To)

## GlazingMaterials

#### WindowMaterial

In [17]:
WindowMaterial_Glazing = pu.parse_idfs(idfs, 'WINDOWMATERIAL:GLAZING', keys)

In [18]:
column_rename = {'Optical_Data_Type':'Optical',
                'Window_Glass_Spectral_Data_Set_Name':'OpticalData',
                'Solar_Transmittance_at_Normal_Incidence':'SolarTransmittance',
                'Front_Side_Solar_Reflectance_at_Normal_Incidence':'SolarReflectanceFront',
                'Back_Side_Solar_Reflectance_at_Normal_Incidence':'SolarReflectanceBack',
                'Infrared_Transmittance_at_Normal_Incidence':'IRTransmittance',
                'Visible_Transmittance_at_Normal_Incidence':'VisibleTransmittance',
                'Front_Side_Visible_Reflectance_at_Normal_Incidence':'VisibleReflectanceFront',
                'Back_Side_Visible_Reflectance_at_Normal_Incidence':'VisibleReflectanceBack',
                'Front_Side_Infrared_Hemispherical_Emissivity':'IREmissivityFront',
                'Back_Side_Infrared_Hemispherical_Emissivity':'IREmissivityBack',
                'Dirt_Correction_Factor_for_Solar_and_Visible_Transmittance':'DirtFactor'}

In [20]:
GlazingMaterials = WindowMaterial_Glazing
GlazingMaterials.rename(columns=column_rename, inplace=True)
GlazingMaterials = GlazingMaterials.drop(['key', 'Solar_Diffusing'], axis=1)
GlazingMaterials['Comment'] = 'default'
GlazingMaterials['Cost'] = 0
GlazingMaterials['DataSource'] = np.NaN
GlazingMaterials['Density'] = 2500
GlazingMaterials['EmbodiedCarbon'] = 0
GlazingMaterials['EmbodiedCarbonStdDev'] = 0
GlazingMaterials['EmbodiedEnergy'] = 0
GlazingMaterials['EmbodiedEnergyStdDev'] = 0
GlazingMaterials['Life'] = 1
GlazingMaterials['SubstitutionRatePattern'] = np.NaN # ! Might have to change to an empty array
GlazingMaterials['SubstitutionTimestep'] = 0
GlazingMaterials['TransportCarbon'] = 0
GlazingMaterials['TransportDistance'] = 0
GlazingMaterials['TransportEnergy'] = 0
GlazingMaterials['Type'] = 'Uncoated' # TODO Further investigation necessary

In [21]:
GlazingMaterials

,Name,Archetype,IREmissivityBack,SolarReflectanceBack,VisibleReflectanceBack,Conductivity,DirtFactor,IREmissivityFront,SolarReflectanceFront,VisibleReflectanceFront,...,EmbodiedCarbonStdDev,EmbodiedEnergy,EmbodiedEnergyStdDev,Life,SubstitutionRatePattern,SubstitutionTimestep,TransportCarbon,TransportDistance,TransportEnergy,Type
$id,,,,,,,,,,,,,,,,,,,,,
0,CLEAR 6MM,NECB 2011-FullServiceRestaurant-NECB HDD Metho...,0.840,0.071,0.080,0.9000,1.0,0.840,0.0710,0.0800,...,0,0,0,1,NaN,0,0,0,0,Uncoated
1,Clear 3mm,NECB 2011-FullServiceRestaurant-NECB HDD Metho...,0.840,0.075,0.081,0.9000,1.0,0.840,0.0750,0.0810,...,0,0,0,1,NaN,0,0,0,0,Uncoated
2,LoE TINT 6MM,NECB 2011-FullServiceRestaurant-NECB HDD Metho...,0.100,0.200,0.054,0.9000,1.0,0.840,0.0930,0.0350,...,0,0,0,1,NaN,0,0,0,0,Uncoated
3,REF D CLEAR 6MM,NECB 2011-FullServiceRestaurant-NECB HDD Metho...,0.820,0.379,0.505,0.9000,1.0,0.840,0.3080,0.4530,...,0,0,0,1,NaN,0,0,0,0,Uncoated
4,REF D TINT 6MM,NECB 2011-FullServiceRestaurant-NECB HDD Metho...,0.820,0.360,0.450,0.9000,1.0,0.840,0.1400,0.1800,...,0,0,0,1,NaN,0,0,0,0,Uncoated
5,Theoretical Glass [167],NECB 2011-Hospital-NECB HDD Method-CAN_PQ_Mont...,0.985,0.000,0.000,2.1073,1.0,0.985,0.7126,0.6988,...,0,0,0,1,NaN,0,0,0,0,Uncoated
6,Theoretical Glass [197],NECB 2011-Hospital-NECB HDD Method-CAN_PQ_Mont...,0.900,0.000,0.000,0.0415,1.0,0.900,0.7151,0.6988,...,0,0,0,1,NaN,0,0,0,0,Uncoated
7,Theoretical Glass [202],NECB 2011-Hospital-NECB HDD Method-CAN_PQ_Mont...,0.900,0.000,0.000,0.0192,1.0,0.900,0.7175,0.6308,...,0,0,0,1,NaN,0,0,0,0,Uncoated
8,Theoretical Glass [207],NECB 2011-Hospital-NECB HDD Method-CAN_PQ_Mont...,0.900,0.000,0.000,0.0133,1.0,0.900,0.6189,0.5100,...,0,0,0,1,NaN,0,0,0,0,Uncoated


In [22]:
GlazingMaterials.index = newrange(GasMaterials, GlazingMaterials)
GlazingMaterials.index.rename('$id', inplace=True)
GlazingMaterials

,Name,Archetype,IREmissivityBack,SolarReflectanceBack,VisibleReflectanceBack,Conductivity,DirtFactor,IREmissivityFront,SolarReflectanceFront,VisibleReflectanceFront,...,EmbodiedCarbonStdDev,EmbodiedEnergy,EmbodiedEnergyStdDev,Life,SubstitutionRatePattern,SubstitutionTimestep,TransportCarbon,TransportDistance,TransportEnergy,Type
$id,,,,,,,,,,,,,,,,,,,,,
2,CLEAR 6MM,NECB 2011-FullServiceRestaurant-NECB HDD Metho...,0.840,0.071,0.080,0.9000,1.0,0.840,0.0710,0.0800,...,0,0,0,1,NaN,0,0,0,0,Uncoated
3,Clear 3mm,NECB 2011-FullServiceRestaurant-NECB HDD Metho...,0.840,0.075,0.081,0.9000,1.0,0.840,0.0750,0.0810,...,0,0,0,1,NaN,0,0,0,0,Uncoated
4,LoE TINT 6MM,NECB 2011-FullServiceRestaurant-NECB HDD Metho...,0.100,0.200,0.054,0.9000,1.0,0.840,0.0930,0.0350,...,0,0,0,1,NaN,0,0,0,0,Uncoated
5,REF D CLEAR 6MM,NECB 2011-FullServiceRestaurant-NECB HDD Metho...,0.820,0.379,0.505,0.9000,1.0,0.840,0.3080,0.4530,...,0,0,0,1,NaN,0,0,0,0,Uncoated
6,REF D TINT 6MM,NECB 2011-FullServiceRestaurant-NECB HDD Metho...,0.820,0.360,0.450,0.9000,1.0,0.840,0.1400,0.1800,...,0,0,0,1,NaN,0,0,0,0,Uncoated
7,Theoretical Glass [167],NECB 2011-Hospital-NECB HDD Method-CAN_PQ_Mont...,0.985,0.000,0.000,2.1073,1.0,0.985,0.7126,0.6988,...,0,0,0,1,NaN,0,0,0,0,Uncoated
8,Theoretical Glass [197],NECB 2011-Hospital-NECB HDD Method-CAN_PQ_Mont...,0.900,0.000,0.000,0.0415,1.0,0.900,0.7151,0.6988,...,0,0,0,1,NaN,0,0,0,0,Uncoated
9,Theoretical Glass [202],NECB 2011-Hospital-NECB HDD Method-CAN_PQ_Mont...,0.900,0.000,0.000,0.0192,1.0,0.900,0.7175,0.6308,...,0,0,0,1,NaN,0,0,0,0,Uncoated
10,Theoretical Glass [207],NECB 2011-Hospital-NECB HDD Method-CAN_PQ_Mont...,0.900,0.000,0.000,0.0133,1.0,0.900,0.6189,0.5100,...,0,0,0,1,NaN,0,0,0,0,Uncoated


#### WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM

In [23]:
WindowMaterialsSimpleGlaze = pu.parse_idfs(idfs, 'WINDOWMATERIAL:SIMPLEGLAZINGSYSTEM', keys)

In [24]:
SimpleGlazing = WindowMaterialsSimpleGlaze.apply(lambda row: pu.simple_glazing(row['Solar_Heat_Gain_Coefficient'], row['UFactor'], row['Visible_Transmittance']), axis=1).apply(pd.Series)
SimpleGlazing.loc[:,'Name'] = WindowMaterialsSimpleGlaze['Name']
SimpleGlazing.loc[:,'DataSource'] = 'EnergyPlus Simple Glazing Calculation'

In [25]:
WindowMaterialsSimpleGlaze.iloc[0]

Name                           Customized Fenestration:: cond=0.220 tvis=0.22...
Archetype                      NECB 2011-FullServiceRestaurant-NECB HDD Metho...
Solar_Heat_Gain_Coefficient                                                  0.6
UFactor                                                                      2.2
Visible_Transmittance                                                       0.21
key                                           WindowMaterial:SimpleGlazingSystem
Name: 0, dtype: object

In [26]:
GlazingMaterials = GlazingMaterials.append(SimpleGlazing, ignore_index=True, sort=True)

## OpaqueMaterials

In [27]:
# MASS
MaterialsMass = pu.parse_idfs(idfs, 'MATERIAL', keys)

In [28]:
# NoMASS
MaterialNoMass_s = []
for idf in idfs:
    MaterialNoMass = idf.idfobjects['MATERIAL:NOMASS']
    MaterialNoMass = [get_values(frame) for frame in MaterialNoMass]
    MaterialNoMass = pd.concat(MaterialNoMass, ignore_index=True, sort=True)
    MaterialNoMass_s.append(MaterialNoMass)
MaterialNoMass = pd.concat(MaterialNoMass_s, keys=keys, names=['Archetype','$id'])
MaterialNoMass = MaterialNoMass.groupby('Name').first()
MaterialNoMass.reset_index(inplace=True)

NameError: name 'get_values' is not defined

In [29]:
OpaqueMaterials = pd.concat([MaterialsMass,MaterialNoMass], sort=True, ignore_index=True)
OpaqueMaterials.index = newrange(GlazingMaterials, OpaqueMaterials)
OpaqueMaterials.index.name = '$id'

TypeError: cannot concatenate object of type "<class 'eppy.idf_msequence.Idf_MSequence'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [ ]:
column_rename = {'Solar_Absorptance':'SolarAbsorptance',
                'Specific_Heat':'SpecificHeat', 
                'Thermal_Absorptance':'ThermalEmittance',
                'Thermal_Resistance':'ThermalResistance',
                'Visible_Absorptance':'VisibleAbsorptance'}

In [ ]:
OpaqueMaterials.rename(columns=column_rename, inplace=True)

In [ ]:
OpaqueMaterials['Comment'] = 'default'
OpaqueMaterials['Cost'] = 0
OpaqueMaterials['DataSource'] = idf.idfobjects['BUILDING'].list2[0][1]
OpaqueMaterials['EmbodiedCarbon'] = 0
OpaqueMaterials['EmbodiedCarbonStdDev'] = 0
OpaqueMaterials['EmbodiedEnergy'] = 0
OpaqueMaterials['EmbodiedEnergyStdDev'] = 0
OpaqueMaterials['Life'] = 1
OpaqueMaterials['MoistureDiffusionResistance'] = 50
OpaqueMaterials['PhaseChange'] = False
OpaqueMaterials['PhaseChangeProperties'] = '' # ! Further investigation needed
OpaqueMaterials['SubstitutionRatePattern'] = np.NaN # ! Might have to change to an empty array
OpaqueMaterials['SubstitutionTimestep'] = 0
OpaqueMaterials['TransportCarbon'] = 0
OpaqueMaterials['TransportDistance'] = 0
OpaqueMaterials['TransportEnergy'] = 0
OpaqueMaterials['Type'] = '' # ! Further investigation necessary
OpaqueMaterials['VariableConductivity'] = False
OpaqueMaterials['VariableConductivityProperties'] = np.NaN # ! Further investigation necessary

In [ ]:
OpaqueMaterials = OpaqueMaterials.drop(['key'], axis=1)

In [ ]:
OpaqueMaterials

## OpaqueConstructions

In [ ]:
# Thermal Resistance : (m2-K)/W
# Conductivity : W/(m-K)

In [ ]:
Constructions = []
for idf in idfs:  
    Construction = idf.idfobjects['CONSTRUCTION']
    Construction = [get_values(frame) for frame in Construction]
    Construction = pd.concat(Construction, ignore_index=True, sort=True)
    Constructions.append(Construction)
    Construction.index = newrange(OpaqueMaterials, Construction)
Construction = pd.concat(Constructions, keys=keys, names=['Archetype','$id'])
Construction = Construction.groupby('Name').first()
Construction.reset_index(inplace=True)

In [ ]:
OpaqueConstructions_s = []
for idf in idfs:
    OpaqueConstructions = idf.idfobjects['BUILDINGSURFACE:DETAILED']
    OpaqueConstructions = [get_values(frame) for frame in OpaqueConstructions]
    OpaqueConstructions = pd.concat(OpaqueConstructions, ignore_index=True, sort=True)
    OpaqueConstructions = OpaqueConstructions.merge(Construction, left_on='Construction_Name', right_on='Name')
    OpaqueConstructions_s.append(OpaqueConstructions)
OpaqueConstructions = pd.concat(OpaqueConstructions_s, keys=keys, names=['Archetype','$id'], sort=True)
OpaqueConstructions = OpaqueConstructions.groupby('Construction_Name').first()
OpaqueConstructions.reset_index(inplace=True)

In [ ]:
OpaqueConstructions

In [ ]:
def label_surface(row):
    """
    This function adds the umi-Category column
    """
    # Floors
    if row['Surface_Type'] == 'Floor':
        if row['Outside_Boundary_Condition'] == 'Surface':
            return 'Interior Floor'
        if row['Outside_Boundary_Condition'] == 'Ground':
            return 'Ground Floor'
        if row['Outside_Boundary_Condition'] == 'Outdoors':
            return 'Exterior Floor'
        else:
            return 'Other'
        
    # Roofs & Ceilings
    if row['Surface_Type'] == 'Roof':
        return 'Roof'
    if row['Surface_Type'] == 'Ceiling':
        return 'Interior Floor'
    # Walls
    if row['Surface_Type'] == 'Wall':
        if row['Outside_Boundary_Condition'] == 'Surface':
            return 'Partition'
        if row['Outside_Boundary_Condition'] == 'Outdoors':
            return 'Facade'
    return 'Other'

In [ ]:
def type_surface(row):
    """
    This function adds the umi-Type column
    """
    # Floors
    if row['Surface_Type'] == 'Floor':
        if row['Outside_Boundary_Condition'] == 'Surface':
            return 3
        if row['Outside_Boundary_Condition'] == 'Ground':
            return 2
        if row['Outside_Boundary_Condition'] == 'Outdoors':
            return 4
        else:
            return np.NaN
        
    # Roofs & Ceilings
    if row['Surface_Type'] == 'Roof':
        return 1
    if row['Surface_Type'] == 'Ceiling':
        return 3
    # Walls
    if row['Surface_Type'] == 'Wall':
        if row['Outside_Boundary_Condition'] == 'Surface':
            return 5
        if row['Outside_Boundary_Condition'] == 'Outdoors':
            return 0
    return np.NaN

In [ ]:
OpaqueConstructions['Category'] = OpaqueConstructions.apply(lambda x: label_surface(x), axis=1)
OpaqueConstructions['Type'] = OpaqueConstructions.apply(lambda x: type_surface(x), axis=1)

In [ ]:
# Lets group by `Construction_Name` to get a list of unique `Surface_Type` and `Outside_Boundary_Condition`.

OpaqueConstructions = OpaqueConstructions.groupby('Construction_Name').first().reset_index()

In [ ]:
def layer_composition(row, df):
    # Assumes 10 max layers
    layers = []
    
    # Let's start with the `Outside_Layer`
    ref, thickness = get_row_prop(row, df, 'Outside_Layer', 'Thickness')
    if thickness:
        layers.append({'Material':{'$ref':ref,'thickness':thickness}})
    else:
        thickness = 0.001 # Very small tickness
        layers.append({'Material':{'$ref':ref,'thickness':thickness}})
    # Then we iterate over the other layers. The number of layers is unknow. Limited to 10 for now
    for i in range(1,10):
        try:
            layer_name = 'Layer_{}'.format(i)
            ref, thickness = get_row_prop(row, df, layer_name, 'Thickness')
            if thickness:
                layers.append({'Material':{'$ref':ref,'thickness':thickness}})
            else:
                thickness = 0.001 # Very small tickness
                layers.append({'Material':{'$ref':ref,'thickness':thickness}})
        except:
            pass #
    return layers

In [ ]:
def get_row_prop(row, df, column_name, prop):
    layer = df.loc[df['Name'] == row[column_name]]
    ref = layer.index[0]
    prop = layer[prop].values[0] # Very small tickness
    return ref, prop

In [ ]:
OpaqueConstructions['Layers'] = OpaqueConstructions.apply(lambda x: layer_composition(x, OpaqueMaterials), axis=1)

In [ ]:
columns = ['Construction_Name', 'Category', 'Layers', 'Type']

In [ ]:
OpaqueConstructions = OpaqueConstructions[columns]

In [ ]:
OpaqueConstructions = OpaqueConstructions.rename(columns={'Construction_Name':'Name'})

In [ ]:
OpaqueConstructions['AssemblyCarbon'] = 0
OpaqueConstructions['AssemblyCost'] = 0
OpaqueConstructions['AssemblyEnergy'] = 0
OpaqueConstructions['Comments'] = 'default'
OpaqueConstructions['DataSource'] = idf.idfobjects['BUILDING'].list2[0][1]
OpaqueConstructions['DisassemblyCarbon'] = 0
OpaqueConstructions['DisassemblyEnergy'] = 0

In [ ]:
OpaqueConstructions.index = newrange(OpaqueMaterials,OpaqueConstructions)
OpaqueConstructions.index.name = '$id'

# WindowConstructions

In [ ]:
WindowConstructions_s = []
for idf in idfs: 
    WindowConstructions = idf.idfobjects['FENESTRATIONSURFACE:DETAILED']
    WindowConstructions = [get_values(frame) for frame in WindowConstructions]
    WindowConstructions = pd.concat(WindowConstructions, ignore_index=True, sort=True)
    WindowConstructions = WindowConstructions.merge(Construction, left_on='Construction_Name', right_on='Name')
    WindowConstructions_s.append(WindowConstructions)
# Groupby `Construction_Name` to keep only unique Window Constructions
WindowConstructions = pd.concat(WindowConstructions_s, keys=keys, names=['Archetype','$id'], sort=True)
WindowConstructions = WindowConstructions.groupby(['Archetype','Construction_Name']).first()
WindowConstructions.index = newrange(OpaqueConstructions,WindowConstructions)
WindowConstructions.index.name = '$id'

In [ ]:
idfs[0].idfobjects['FENESTRATIONSURFACE:DETAILED']

# DaySchedules

In [ ]:
from datetime import datetime, timedelta

In [ ]:
DaySchedules = []
for idf in idfs:
    DaySchedule = idf.idfobjects['SCHEDULE:DAY:INTERVAL']
    DaySchedule = [get_values(frame) for frame in DaySchedule]
    DaySchedule = pd.concat(DaySchedule, ignore_index=True, sort=True)
    DaySchedules.append(DaySchedule)
DaySchedule = pd.concat(DaySchedules, keys=keys, names=['Archetype','$id'], sort=True)
DaySchedule = DaySchedule.groupby('Name').first()
DaySchedule.reset_index(inplace=True)
DaySchedule.index = newrange(WindowConstructions, DaySchedule)
DaySchedule.index.name = '$id'

In [ ]:
def my_to_datetime(date_str):
    if date_str[0:2] != '24':
        return datetime.strptime(date_str, '%H:%M') - timedelta(hours=1)
    return datetime.strptime('23:00', '%H:%M')

In [ ]:
def time2time(row):
    time_seg = []
    for i in range(1,25):
        time = row['Time_{}'.format(i)] # Time_i
        value = row['Value_Until_Time_{}'.format(i)] # Value_Until_Time_i
        if str(time) != 'nan' and str(value) != 'nan':
#             print(time)
            time = my_to_datetime(time).hour
#             print(time)
            times = np.ones(time+1) * float(value)
            time_seg.append(times)
    arrays = time_seg
    array = time_seg[0]
    length = len(arrays[0])
    for i, a in enumerate(arrays):
        if i != 0:
            array = np.append(array, a[length-1:-1])
            length = len(a)
    return array

In [ ]:
DaySchedule['Values'] = DaySchedule.apply(lambda x: time2time(x), axis=1)

In [ ]:
DaySchedule = DaySchedule.loc[:,['Name', 'Values']]
DaySchedule.loc[:,'Category'] = 'Day'
DaySchedule.loc[:,'Comments'] = 'Comments'
DaySchedule.loc[:,'DataSource'] = 'default'
DaySchedule.loc[:,'Type'] = 'Fraction'
DaySchedule

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Plotting the schedules
for aid, grp in DaySchedule.groupby(['Name']):
    plt.plot(grp['Values'].values[0]) 

# WeekSchedule

In [ ]:
WeekSchedules = []
for idf in idfs:
    WeekSchedule = idf.idfobjects['SCHEDULE:WEEK:DAILY']
    WeekSchedule = [get_values(frame) for frame in WeekSchedule]
    WeekSchedule = pd.concat(WeekSchedule, ignore_index=True, sort=True)
    WeekSchedules.append(WeekSchedule)
WeekSchedule = pd.concat(WeekSchedules, keys=keys, names=['Archetype','$id'], sort=True)
WeekSchedule = WeekSchedule.groupby('Name').first()
WeekSchedule.reset_index(inplace=True)

In [ ]:
WeekSchedule

In [ ]:
def schedule_composition(row, df):
    # Assumes 7 days
    day_schedules = []
    days = ['Monday_ScheduleDay_Name',
           'Tuesday_ScheduleDay_Name',
           'Wednesday_ScheduleDay_Name',
           'Thursday_ScheduleDay_Name',
           'Friday_ScheduleDay_Name',
           'Saturday_ScheduleDay_Name',
           'Sunday_ScheduleDay_Name'] # With weekends last (as defined in umi-template)
    # Let's start with the `Outside_Layer`
    for day in days:
        try:
            ref, day_schedule = get_row_prop(row, df, day, 'Values')
            day_schedules.append({'$ref':ref})
        except:
            pass
    return day_schedules

In [ ]:
WeekSchedule['Values'] = WeekSchedule.apply(lambda x: schedule_composition(x, DaySchedule), axis=1)

In [ ]:
WeekSchedule = WeekSchedule[['Name','Values']]
WeekSchedule['Category'] = 'Week'
WeekSchedule['Comments'] = 'default'
WeekSchedule['DataSource'] = idf.idfobjects['BUILDING'].list2[0][1]

In [ ]:
WeekSchedule.index = newrange(DaySchedule, WeekSchedule)
WeekSchedule.index.name = '$id'
WeekSchedule

# YearSchedules

In [ ]:
YearSchedules = []
for idf in idfs:
    YearSchedule = idf.idfobjects['SCHEDULE:YEAR']
    YearSchedule = [get_values(frame) for frame in YearSchedule]
    YearSchedule = pd.concat(YearSchedule, ignore_index=True, sort=True)
    YearSchedules.append(YearSchedule)
YearSchedule = pd.concat(YearSchedules, keys=keys, names=['Archetype','$id'], sort=True)
YearSchedule = YearSchedule.groupby('Name').first()
YearSchedule.reset_index(inplace=True)

In [ ]:
def year_parts(row, df):
    ref, prop = get_row_prop(row, df, 'ScheduleWeek_Name_1', 'Name')
    fromday = row['Start_Day_1']
    frommonth = row['Start_Month_1']
    today = row['End_Day_1']
    tomonth = row['End_Month_1']
    parts = {'FromDay': fromday,
             'FromMonth': frommonth,
             'Schedule': {'$ref': ref},
             'ToDay': today,
             'ToMonth': tomonth}
    return parts

In [ ]:
YearSchedule['Parts'] = YearSchedule.apply(lambda x: year_parts(x, WeekSchedule), axis=1)

In [ ]:
YearSchedule

In [ ]:
YearSchedule = YearSchedule[['Name', 'Schedule_Type_Limits_Name','Parts']]

YearSchedule['Category'] = 'Year'
YearSchedule['Comments'] = 'default'
YearSchedule['DataSource'] = idf.idfobjects['BUILDING'].list2[0][1]

YearSchedule.index = newrange(WeekSchedule, YearSchedule)
YearSchedule.index.name = '$id'
YearSchedule

# Start of json READ

At this point, the needed information is easier to acces in the `qacq` files (json format)

In [ ]:
import json

In [ ]:
files = glob.glob("data/necb/NECB_2011_Montreal/*.json")

In [ ]:
# Reading the json as a dict
qaqcs = []
for file in files:
    with open(file) as json_data:
        qaqcs.append(json.load(json_data))

all_qaqcs = {}
for qaqc in qaqcs:
    qaqc_dfs = {}
    name = qaqc['building']['name']
    for key in qaqc.keys():
        if isinstance(qaqc[key], dict):
            try:
                qaqc_dfs[key] = pd.DataFrame.from_dict(qaqc[key])
            except Exception as inst:
                qaqc_dfs[key] = pd.DataFrame.from_dict([qaqc[key]])
        elif isinstance(qaqc[key], list):
            qaqc_dfs[key] = pd.DataFrame.from_dict(qaqc[key])
        else:
            qaqc_dfs[key] = qaqc[key]
    all_qaqcs[name] = qaqc_dfs

# Zones

Zones can be identified by the term `horzontal_placement` (norht, south, east, west, core) in the NECB building definition [https://github.com/NREL/OpenStudio-Prototype-Buildings/blob/master/lib/btap/measures/btap_equest_converter/compliance.rb]

In [ ]:
def iscore(row):
    """
    Helps to group by core and perimeter zones
    """
    if 'core' in row['thermal_zone'].lower(): # We look for the string `core` in the Zone_Name
        return 'Core'
    else:
        return 'Perimeter'

In [ ]:
Zones = []
for qaqc in all_qaqcs.values():
    Zone = qaqc['spaces']
    Zones.append(Zone)
Zones = pd.concat(Zones, keys=all_qaqcs.keys(), names=['Archetype','$id'], sort=True)
Zones['Space_Type'] = Zones.apply(lambda x: iscore(x), axis=1)

In [ ]:
def ach(row):
    """
    Calculates Air changes per hour (ACH)
    """
    q = row['infiltration_flow_m3_per_s']
    V = row['volume']
    if q > 0:
        return 3600 * q / V
    return np.NaN

In [ ]:
# Calculating the infiltration flow by use of the area-infiltration flow times the exterior wall area
Zones['infiltration_flow_m3_per_s'] = Zones['infiltration_flow_per_m2'] * Zones['exterior_wall_area']

In [ ]:
Zones['air_changes_per_hour'] = Zones.apply(lambda x: ach(x), axis=1)

In [ ]:
# Let's add the space area

spacetype_area_breakdown = []
for qaqc in all_qaqcs.values():
    area = qaqc['spacetype_area_breakdown'].T.rename(columns={0:'Space_Area'})
    spacetype_area_breakdown.append(area)
spacetype_area_breakdown = pd.concat(spacetype_area_breakdown, axis=0, keys=all_qaqcs.keys(), names=['Archetype','space_type_name'], sort=True)

In [ ]:
Zones['space_type_name'] = Zones['space_type_name'].str.replace(' ','_').str.lower()

In [ ]:
Zones = Zones.reset_index().set_index(['Archetype','space_type_name']).merge(spacetype_area_breakdown, left_index=True, right_index=True, how='left')
Zones = Zones.reset_index().set_index(['Archetype','$id'])

In [ ]:
Zones

In [ ]:
# Let's add a the `waterUseEquipment` as columns instead of the object they are in the DataFrame.
# To do this, we merge a new dataframe that is created with the apply(pd.Series) function. This transforms
# the dict stucture of each rows into a the differetn columns. The apply the Pd.Series fucntion twice because
# the dicts are inside a list.

Zones = Zones.join(Zones['waterUseEquipment'].apply(pd.Series)[0].apply(pd.Series), sort=True)

In [ ]:
def weighted_mean(series):
    """
    Evaluates a weighteed average while ignoring NaNs
    """
    index = ~np.isnan(series)
    if np.any(index):
        weights=Zones.loc[series.index, 'volume'] * Zones.loc[series.index, 'multiplier']
        weights=weights[index]
        a = series[index]
        return np.average(a, weights=weights)
    return 0

In [ ]:
# Define a lambda function to compute the weighted mean:
# wm = lambda x: np.average(~np.isnan(x), weights=(Zones.loc[x.index, 'volume'] * Zones.loc[x.index, 'multiplier'])[~np.isnan(x)])
wu = lambda x: x.apply(pd.Series)
# Define a dictionary with the functions to apply for a given column:
f = {'air_changes_per_hour': {'weighted_mean' : weighted_mean},
     'occ_per_m2' : {'weighted_mean' : weighted_mean},
     'breathing_zone_outdoor_airflow_vbz' : {'weighted_mean' : weighted_mean},
     'electric_w_per_m2' : {'weighted_mean' : weighted_mean},
     'lighting_w_per_m2' : {'weighted_mean' : weighted_mean},
     'peak_flow_rate_per_area' : {'weighted_mean' : weighted_mean},
     'Space_Area' : {'sum' : 'sum'}
    }

# Groupby and aggregate with your dictionary:
ZonesByType = Zones.groupby(['Archetype','Space_Type'], sort=True).agg(f).reset_index()
ZonesByType['FlowRatePerFloorArea'] = ZonesByType['peak_flow_rate_per_area'] * 36000 # m3/s/m2 to m3/h/m2
ZonesByType

# DomesticHotWaterSettings

In [ ]:
# Category
# DataSource
# FlowRatePerFloorArea
# IsOn
# Name
# WaterSchedule.$ref - Source: Probably in idf file
# WaterSupplyTemperature
# WaterTemperatureInlet - Source: Probably in idf file
DomesticHotWaterSettingsColumns = ['Category', 'DataSource', 'FlowRatePerFloorArea', 'IsOn', 'Name',
       'WaterSchedule.$ref', 'WaterSupplyTemperature',
       'WaterTemperatureInlet']
# Creating a DataFrame

possibleSchedules = DaySchedule[DaySchedule.Name.str.contains('Service Water Loop Temp', case=False)]

# WaterSchedule.$ref
WaterSchedule_ref = YearSchedule[YearSchedule.Name.str.contains('Service Water Loop Temp', case=False)].index

# WaterSupplyTemperature
waterschedule = DaySchedule.reset_index().set_index('Name').loc[possibleSchedules.Name.values,:]
WaterSupplyTemperature = waterschedule.Values.mean().mean() # the first mean() averages possible multiple schedules, the second mean(gets the value

# WaterTemperatureInlet - Source: Probably in idf file
WaterTemperatureInlet = 8.0 # ! This needs to be investigated

DomesticHotWaterSettings = ZonesByType.loc[:,['Archetype', 'Space_Type','FlowRatePerFloorArea']]
DomesticHotWaterSettings.loc[:,'Category'] = DomesticHotWaterSettings['Space_Type']
DomesticHotWaterSettings.loc[:,'IsOn'] = DomesticHotWaterSettings.apply(lambda x: x['FlowRatePerFloorArea'] > 0, axis=1)
DomesticHotWaterSettings.loc[:,'Name'] = DomesticHotWaterSettings['Archetype'] + '_' + DomesticHotWaterSettings['Space_Type']
DomesticHotWaterSettings.loc[:,'DataSource'] = DomesticHotWaterSettings['Archetype']
DomesticHotWaterSettings.loc[:,'WaterSchedule.$ref'] = WaterSchedule_ref.values[0]
DomesticHotWaterSettings.loc[:,'WaterSupplyTemperature'] = WaterSupplyTemperature
DomesticHotWaterSettings.loc[:,'WaterTemperatureInlet'] = WaterTemperatureInlet
DomesticHotWaterSettings.index = newrange(YearSchedule, DomesticHotWaterSettings)
DomesticHotWaterSettings.index.name = '$id'
DomesticHotWaterSettings = DomesticHotWaterSettings[DomesticHotWaterSettingsColumns]

In [ ]:
DomesticHotWaterSettings

# VentilationSettings

In [ ]:
# 'Afn',
# 'Category',
# 'DataSource',
# 'Infiltration',
# 'IsBuoyancyOn',
# 'IsInfiltrationOn',
# 'IsNatVentOn',
# 'IsScheduledVentilationOn',
# 'IsWindOn',
# 'Name',
# 'NatVentMaxOutdoorAirTemp',
# 'NatVentMaxRelHumidity',
# 'NatVentMinOutdoorAirTemp',
# 'NatVentSchedule.$ref',
# 'NatVentZoneTempSetpoint',
# 'ScheduledVentilationAch',
# 'ScheduledVentilationSchedule.$ref',
# 'ScheduledVentilationSetpoint'

VentilationSettingsSettingsColumns = ['Afn', 'Category', 'DataSource', 'Infiltration', 'IsBuoyancyOn',
       'IsInfiltrationOn', 'IsNatVentOn', 'IsScheduledVentilationOn',
       'IsWindOn', 'Name', 'NatVentMaxOutdoorAirTemp', 'NatVentMaxRelHumidity',
       'NatVentMinOutdoorAirTemp', 'NatVentSchedule.$ref',
       'NatVentZoneTempSetpoint', 'ScheduledVentilationAch',
       'ScheduledVentilationSchedule.$ref', 'ScheduledVentilationSetpoint']

In [ ]:
VentilationSettings = ZonesByType.loc[:,['Archetype', 'Space_Type']]

In [ ]:
VentilationSettings.loc[:,'Infiltration'] = ZonesByType.loc[:,'air_changes_per_hour'].values
VentilationSettings.loc[:,'IsBuoyancyOn'] = True
VentilationSettings.loc[:,'IsInfiltrationOn'] = VentilationSettings.apply(lambda x: x.loc['Infiltration'] > 0, axis=1)
VentilationSettings.loc[:,'IsNatVentOn'] = False
VentilationSettings.loc[:,'IsScheduledVentilationOn'] = False
VentilationSettings.loc[:,'IsWindOn'] = False
VentilationSettings.loc[:,'Name'] = VentilationSettings['Archetype'] + '_' + VentilationSettings['Space_Type']
VentilationSettings.loc[:,'NatVentMaxOutdoorAirTemp'] = 26.0
VentilationSettings.loc[:,'NatVentMaxRelHumidity'] = 80.0
VentilationSettings.loc[:,'NatVentMinOutdoorAirTemp'] = 18.0

In [ ]:
VentilationSettings

In [ ]:
YearSchedule